In [7]:
import cv2
video_path = 0
cap = cv2.VideoCapture(video_path)

import mediapipe as mp

gesture = {
    0:'fist', 1:'one', 2:'two', 3:'three', 4:'four', 5:'five',
    6:'six', 7:'rock', 8:'spiderman', 9:'yeah', 10:'ok',
}
rsp_gesture = {0:'rock', 5:'paper', 9:'scissors'}
# 이미지에서 손 찾기 기능 불러오기
mp_hands = mp.solutions.hands
# 찾은 손 관절 이미지에 표현하기
mp_drawing = mp.solutions.drawing_utils
# 손 찾기 기능 세부 설정
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 제스처 인식 모델 학습하기
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
file = np.genfromtxt('img/gesture_train.csv',delimiter=',')
angle = file[:,:-1].astype(np.float32)
label = file[:,-1].astype(np.float32)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(angle,label)

# 손 인식하기


while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    result = hands.process(frame)
    if result.multi_hand_landmarks is not None:
        rsp_result = []
        # 21개 관절을 하나씩 그리고 연결하기
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # joint들로 관절값 구하기
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v 유클리디안 길이
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # 관절값으로 관절 각도 구하기
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # radian각도를 degree각도로 변경하기

            # 제스쳐 인식시키기
            data = np.array([angle], dtype=np.float32)
            results = knn.predict(data)
            idx = int(results)
        
            # 제스쳐 인식되면 표시하기
            if idx in rsp_gesture.keys():
                org = (int(res.landmark[0].x * frame.shape[1]), int(res.landmark[0].y * frame.shape[0]))
                cv2.putText(frame, text=rsp_gesture[idx].upper(), org=(org[0], org[1] + 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                rsp_result.append({
                    'rsp': rsp_gesture[idx],
                    'org': org
                })
            
            mp_drawing.draw_landmarks(frame, res, mp_hands.HAND_CONNECTIONS)
            
             # Who wins?
            if len(rsp_result) == 2:
                winner = None
                text = ''

                if rsp_result[0]['rsp']=='rock':
                    if rsp_result[1]['rsp']=='rock'     : text = 'Tie'
                    elif rsp_result[1]['rsp']=='paper'  : text = 'Paper wins'  ; winner = 1
                    elif rsp_result[1]['rsp']=='scissors': text = 'Rock wins'   ; winner = 0
                elif rsp_result[0]['rsp']=='paper':
                    if rsp_result[1]['rsp']=='rock'     : text = 'Paper wins'  ; winner = 0
                    elif rsp_result[1]['rsp']=='paper'  : text = 'Tie'
                    elif rsp_result[1]['rsp']=='scissors': text = 'Scissors wins'; winner = 1
                elif rsp_result[0]['rsp']=='scissors':
                    if rsp_result[1]['rsp']=='rock'     : text = 'Rock wins'   ; winner = 1
                    elif rsp_result[1]['rsp']=='paper'  : text = 'Scissors wins'; winner = 0
                    elif rsp_result[1]['rsp']=='scissors': text = 'Tie'

                if winner is not None:
                    cv2.putText(frame, text='Winner', org=(rsp_result[winner]['org'][0], rsp_result[winner]['org'][1] + 70), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0, 255, 0), thickness=3)
                cv2.putText(frame, text=text, org=(int(frame.shape[1] / 3), 100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0, 0, 255), thickness=3)
    cv2.imshow('VideoFrame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
